为了调用父类（超类）的一个方法，可以使用 `super()` 函数。例如：

In [2]:
class A:
    def spam(self):
        print("A.spam")


class B(A):
    def spam(self):
        print("B.spam")
        super().spam()

`super()` 函数的一个常见用法是 *** 用在 `__init__()` 方法中确保父类被正确的初始化 ***

In [5]:
class A:
    def __init__(self):
        self.x = 0
        
class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

In [7]:
b = B()
print (b.y)
print (b.x)

1
0


super() 函数的另外一个常用方式是：*** 覆盖python特殊方法的代码 ***

In [8]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj
        
    # 委托内部实例查找属性
    def __getattr__(self, name):
        return getattr(self._obj, name)
    
    # 委托属性分配
    def __setattr__(self, name, value):
        if name.startswith('_'):
            # 调用原始的 __setattr__()方法
            super().__setattr__(name, value)
        else:
            setattr(self._obj, name, value)
        

在上面的代码中， `__setattr__()`的实现包含一个名字检查。如果某个属性名以下划线开头，就通过`super()`调用原始的 `__setattr__()`，否则就委派给内部的代理对象去处理。

super() 即便没有显式的指明某个类的父类， super() 仍然可以有效的工作

####  正确的使用 super() 方法

In [10]:
class Base:
    def __init__(self):
        print("Base.__init__")
        
class A(Base):
    def __init__(self):
        super().__init__()
        print("A.__init__")

a = A()

Base.__init__
A.__init__


尽管对于大部分代码而言上面的做法没有什么问题，但是在更加复杂的涉及到**多继承**的代码中，就有可能导致奇怪的问题。例如：

In [12]:
class Base:
    def __init__(self):
        print("Base.__init__")
        
class A(Base):
    def __init__(self):
        Base.__init__(self)
        print("A.__init__")

class B(Base):
    def __init__(self):
        Base.__init__(self)
        print("B.__init__")

class C(A,B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
        print("C.__init__")
        
c = C()
    

Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


可以看到`Base.__init__()`被调用了两次，可能两次调用没有什么坏处，但有时候确不是这样。  
另一方面，如果在代码中换成使用`super()`，结果就很完美了：

In [13]:
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')

class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')

class C(A,B):
    def __init__(self):
        super().__init__()  # Only one call to super() here
        print('C.__init__')

c = C()

Base.__init__
B.__init__
A.__init__
C.__init__


为了了解其中的缘由，需要花点时间去了解 ↓ 
####  python 是如何实现继承的

对于开发人员定义的任何一个类，Python会计算出一个**方法解析顺序列表（MRO  Method Resolution Order）** 。对于任何支持继承的编程语言来说，其方法可能定义在当前类，也可能来自基类，所以在方法调用时就需要对当前类和基类进行搜索以确定方法所在的位置。MRO就是其搜索的顺序。对于只支持单继承的语言来说，MRO一般比较简单，而对于Python这种支持多继承的语言来说，MRO就会复杂很多

<img src=https://hanjianwei.com/assets/2013-07-25-python-mro/class_diamond.svg>

MRO 是把类关系线性化的一个过程，线性化方式决定了程序运行过程中具体会调用哪个方法。   
Python至少有三种MRO：   
1、经典类（classic class）的深度遍历   
2、python2.2的新式类（new-style class）与计算    
3、python2.3的新式类的***C3算法***，他也是python3唯一支持的方法   

In [17]:
C.__mro__

(__main__.C, __main__.A, __main__.B, __main__.Base, object)

为了实现继承，Python会在MRO列表上从左到右开始查找基类，知道找到第一个匹配这个属性的类位置。   
这个MRO的列表的构造是根据 **C3线性化算法** 来实现的，该算法实际上就是**合并所有父类MRO列表**，并遵循如下三条准则：  
1、子类会先于父类检查   
2、多个父类会根据他们在列表中的顺序被减产   
3、如果对下一个类存在两个合法的选择，选择第一个父类  

***总的来说，MRO列表中的类会让你定义的任意类的层级关系变得有意义 ***

当使用super() 函数的时候， python会在MRO上继续搜索下一个类，只要每个重定义的方法统一使用super() 并只调用它一次，那么控制流最终会遍历完整个MRO列表，每个方法也只会被调用一次。
可以把MRO理解为一个类的关系栈，super()为栈的遍历指针

super()一个有趣的点是它并不一定去查找某个类在MRO中的下一个直接父类，甚至可以在一个没有父类的类中使用它。例如：

In [25]:
class A:
    def spam(self):
        print ("A.spam")
        super().spam()

a = A()
print(A.__mro__)
a.spam()

(<class '__main__.A'>, <class 'object'>)
A.spam


AttributeError: 'super' object has no attribute 'spam'

直接调用时会报错，这是因为MRO中没有父类具有spam这个方法，只有一个object类

但是如果使用多继承的话就会出现有意思的事：

In [27]:
class A:
    def spam(self):
        print ("A.spam")
        super().spam()

class B:
    def spam(self):
        print("B.spam")
        
class C(A,B):
    pass

c = C()
c.spam()
print(C.__mro__)

A.spam
B.spam
(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)


可以看到在类A中使用 super().spam() 实际上调用的是和A毫无关系的B类的spam() 方法，这是因为在检索C的MRO队列时，会先检索A ，然后检索B，然后才是object基类

其实在定义混入类的时候，super()是非常普遍的。  
然而由于super()可能调用不是你所想要的方法，所以应该尊徐一些通用原则：  
1、确保在继承体系中所有相同名字的方法拥有**可兼容的参数签名**（例如相同的参数个数和参数名称）  
2、最好确保最顶层的类提供了这个方法的实现，这样的话在MRO上面的查找可以找到某个确定的方法